In [24]:
%matplotlib inline
import pandas as pd
from IPython.core.display import HTML
import numpy as np
import matplotlib.pyplot as plt

In [25]:
settle = pd.DataFrame.from_csv('settle_full.csv', index_col=None)
fydata = pd.DataFrame.from_csv('fydata.csv', index_col=None)

ABRT = pd.DataFrame.from_csv('ABRT.csv',index_col=None)

In [26]:
settle = settle.rename(columns={'Month':'month'})
settle['year'] = pd.DatetimeIndex(settle['classend']).year
fydata = fydata.rename(columns={'datadate':'date'})
fydata['date'] = pd.to_datetime(fydata['date'].astype(str), format='%m/%d/%Y')
fydata['year'] = pd.DatetimeIndex(fydata['date']).year
fydata['month'] = pd.DatetimeIndex(fydata['date']).month
fydata = fydata.rename(columns={'month':'f_month'})
m_fydata = fydata[['tic', 'year','f_month']]

In [27]:
fydata['wcap_1'] = fydata.wcap.shift(1)
fydata['dwcap'] = fydata.wcap - fydata.wcap_1
fydata['ebit_1'] = fydata.ebit.shift(1)
fydata['fcf'] = fydata.ebit - fydata.txpd + fydata.dp - fydata.dwcap - fydata.capx
fydata['fcf_1'] = fydata.fcf.shift(1)
fydata['size'] = np.log(fydata['mkvalt'])

fydata.dt = fydata.dt.replace({ 0 : np.nan })


fydata['debtratio'] = fydata['dltt'].div(fydata['at'], axis=0)

fydata['M2B'] = fydata['mkvalt'].div(fydata['at'], axis=0)

In [28]:
m1_settle = pd.merge(settle, m_fydata, how='left', on=['tic', 'year'])

m1_settle = m1_settle.rename(columns={'year':'ceyr'})

def yearcheck (row):
    if row ['month']>row ['f_month']:
        return row['ceyr'] + 1
    return row['ceyr']

m1_settle['year'] = m1_settle.apply (lambda row: yearcheck (row),axis=1)

m1_settle['styr'] = pd.DatetimeIndex(m1_settle['settledate']).year

def yearchecks (row):
    if row ['month']>row ['f_month']:
        return row['styr'] + 1
    return row['styr']

m1_settle['styr'] = m1_settle.apply (lambda row: yearchecks (row),axis=1)
#Merge finished for FI and Settle

In [29]:
m1_settle = m1_settle.drop_duplicates(subset = ['number'])

In [30]:
# class end date data set
me_fi_ce = pd.merge(m1_settle, fydata, how = 'left', on = ['tic', 'year'])

me_fi_ce = me_fi_ce.rename(columns={'at':'at_ce', 'capx':'capx_ce', 'ceq':'ceq_ce','revt':'revt_ce',
       'dltt':'dltt_ce', 'dp':'dp_ce', 'dt':'dt_ce', 'optrfr':'optrfr_ce', 'setp':'setp_ce', 'txpd':'txpd_ce', 'wcap':'wcap_ce',
       'xintd':'xintd_ce', 'mkvalt':'mkvalt_ce', 'wcap_1':'wcap_1_ce', 'dwcap':'dwcap_ce', 'fcf':'fcf_ce','fcf_1':'fcf_1_ce','size':'size_ce', 
        'debtratio':'debtratio_ce','M2B':'M2B_ce', 'at_log':'at_log_ce'})


me_fi_ce = me_fi_ce.drop(['month', 'gap', 'sic_x','tic.1',
       'lgap', 'f_month_x', 'year', 'gvkey', 'date','sic_y', 'f_month_y',], axis=1)

In [31]:
me_fi_ce = me_fi_ce.drop_duplicates(subset = ['tic','settledate'])

In [32]:
# settlemt date dataset
m1_settle['year'] = pd.DatetimeIndex(m1_settle['settledate']).year

me_fi_st = pd.merge(m1_settle, fydata, how = 'left', on = ['tic', 'year'])

me_fi_st = me_fi_st.rename(columns={'at':'at_st', 'capx':'capx_st', 'ceq':'ceq_st','revt':'revt_st',
       'dltt':'dltt_st', 'dp':'dp_st', 'dt':'dt_st', 'optrfr':'optrfr_st', 'setp':'setp_st', 'txpd':'txpd_st', 'wacp':'wcap_st',
       'xintd':'xintd_st', 'mkvalt':'mkvalt_st', 'wcap_1':'wcap_1_st', 'dwcap':'dwcap_st', 'fcf':'fcf_st','fcf_1':'fcf_1_st','size':'size_st', 
        'debtratio':'debtratio_st','M2B':'M2B_st', 'at_log':'at_log_st'})

me_fi_st = me_fi_st.drop(['sic_x','f_month_x','tic.1','case', 'classification',
       'Accounting', 'Secondary Market & Trading', 'Financial Intermediatary',
       'Non-Accounting disclsoure and misrepresentation', 'Others',], axis=1)

me_fi_st = me_fi_st.rename(columns={'sic_y':'sic','f_month_y':'f_month'})
me_fi_st= me_fi_st.drop_duplicates(subset = ['number'])

In [33]:
ABRT = pd.DataFrame.from_csv('ABRT.csv', index_col=None)

ABRT = ABRT.drop_duplicates(subset=['ticker', 'evtdate'])

ABRT['evtdate'] = pd.to_datetime(ABRT['evtdate'] .astype(str), format='%d%b%Y')



In [34]:
me_fi_st.columns

Index(['number', 'tic', 'settledate', 'classend', 'ceyr', 'month', 'gap',
       'lgap', 'settleamt', 'year', 'styr', 'gvkey', 'date', 'fyear', 'at_st',
       'capx_st', 'ceq_st', 'dltt_st', 'dp_st', 'dt_st', 'ebit', 'revt_st',
       'setp_st', 'txpd_st', 'wcap', 'xintd_st', 'mkvalt_st', 'sic', 'f_month',
       'wcap_1_st', 'dwcap_st', 'ebit_1', 'fcf_st', 'fcf_1_st', 'size_st',
       'debtratio_st', 'M2B_st'],
      dtype='object')

In [35]:
me_fi_ce= me_fi_ce.drop(['classend', 'ceyr', 'settleamt','styr','case'], axis=1)

In [36]:


me_fi_ce = me_fi_ce.rename(columns={'fyear':'fyear_ce','ebit':'ebit_ce','ebit_1':'ebit_1_ce'})
me_fi_st= me_fi_st.rename(columns={'fyear':'fyear_st','ebit':'ebit_st','ebit_1':'ebit_1_st'})



mefi = pd.merge (me_fi_st, me_fi_ce, how = "inner", on =['number','tic','settledate'])

mefi['settledate'] = pd.to_datetime(mefi['settledate'] .astype(str), format="%m/%d/%Y")
mefi['classend'] = pd.to_datetime(mefi['classend'] .astype(str), format="%m/%d/%Y")






ABRT = ABRT [['ticker','evtdate','cret','car','bhar']]

ABRT [['cret','car','bhar']] = ABRT [['cret','car','bhar']]*100

mefi['styr'] = pd.DatetimeIndex(mefi['settledate']).year
mefi['ceyr'] = pd.DatetimeIndex(mefi['classend']).year

ABRT = ABRT.rename(columns={'ticker':'tic','evtdate':'settledate'})

settledata = pd.merge (ABRT, mefi, how = "inner", on =['tic','settledate'])


In [37]:
settledata = settledata.drop_duplicates(subset = ['tic','settledate'])

In [38]:
settledata.to_csv('settledata.csv', encoding='utf-8',index=False)

In [39]:
settledata.columns

Index(['tic', 'settledate', 'cret', 'car', 'bhar', 'number', 'classend',
       'ceyr', 'month', 'gap', 'lgap', 'settleamt', 'year', 'styr', 'gvkey',
       'date', 'fyear_st', 'at_st', 'capx_st', 'ceq_st', 'dltt_st', 'dp_st',
       'dt_st', 'ebit_st', 'revt_st', 'setp_st', 'txpd_st', 'wcap', 'xintd_st',
       'mkvalt_st', 'sic', 'f_month', 'wcap_1_st', 'dwcap_st', 'ebit_1_st',
       'fcf_st', 'fcf_1_st', 'size_st', 'debtratio_st', 'M2B_st',
       'classification', 'Accounting', 'Secondary Market & Trading',
       'Financial Intermediatary',
       'Non-Accounting disclsoure and misrepresentation', 'Others', 'fyear_ce',
       'at_ce', 'capx_ce', 'ceq_ce', 'dltt_ce', 'dp_ce', 'dt_ce', 'ebit_ce',
       'revt_ce', 'setp_ce', 'txpd_ce', 'wcap_ce', 'xintd_ce', 'mkvalt_ce',
       'wcap_1_ce', 'dwcap_ce', 'ebit_1_ce', 'fcf_ce', 'fcf_1_ce', 'size_ce',
       'debtratio_ce', 'M2B_ce'],
      dtype='object')